# Tensorflow ImageNet-ResNet50 Patch Demo

This notebook demonstrates the adversarial patch attack applied on the ResNet50 model.

The following two sections cover experiment setup and is similar across all demos.


## Setup: Experiment Name and Fruits360 Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

### Important: Users will need to verify or update the following parameters:
- Ensure that the `USERNAME` parameter is set to your own name."
- Ensure that the `DATASET_DIR` parameter is set to the location of the ImageNet dataset directory. Currently set to `/nfs/data/ImageNet-Kaggle-2017"` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "howard"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Ensure that the base dataset location is properly set here.
DATASET_DIR = "/nfs/data/ImageNet-Kaggle-2017"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_imagenet_adversarial_patch"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [2]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

Available rules:

clean               Remove temporary files 
data                Download and prepare MNIST dataset 
initdb              Initialize the RESTful API database 
services            Launch the Minio S3 and MLFlow Tracking services 
teardown            Destroy service containers 
workflows           Create workflows tarball 


The ImageNet dataset needed for the demo is available in all worker containers at the path `/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/`.
The following directores contain labeled JPEG images:

    train  
    val-sorted  
    val-sorted-1000
    val-sorted-5000
    val-sorted-10000  
    
Both train and val-sorted contain a full set of labeled training and test images. val-sorted-# directories hold a random partion of labeled images, with 1000, 5000, or 10000 images in total. 

   Each of the directories listed follows the general subfolder structure below:

    ImageNet
    │ 
    │ 
    ├── train
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    │ 
    └── val-sorted
    │ 
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}

The subfolders under `Mnist/training/` and `Mnist/testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [3]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [4]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [5]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'createdOn': '2021-01-06T20:00:36.372309',
 'experimentId': 20,
 'lastModified': '2021-01-06T20:00:36.372309',
 'name': 'howard_imagenet_adversarial_patch'}

In [6]:
# The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
# The run ID is needed to link dependencies between jobs.
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]

def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response

def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response

# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run

def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

Next, we need to train our model.
We will be using the V100 GPUs that are available on the DGX Workstation, which we can use by submitting our job to the `"tensorflow_gpu"` queue.
We will train the ResNet50 model on a subset of the ImageNet dataset.,

In [7]:
response_resnet50_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        "-P model_architecture=resnet50",
        f"-P data_dir={DATASET_DIR}/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Initialization job for ResNet50 neural network submitted")
print("")
pprint.pprint(response_resnet50_train)

response_resnet50_train = get_run_id(response_resnet50_train)

Initialization job for ResNet50 neural network submitted

{'createdOn': '2021-02-14T21:13:14.365622',
 'dependsOn': None,
 'entryPoint': 'init_model',
 'entryPointKwargs': '-P batch_size=20 -P model_architecture=resnet50 -P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000',
 'experimentId': 20,
 'jobId': '5e30b0eb-de4c-4b5c-a949-1fe044466845',
 'lastModified': '2021-02-14T21:13:14.365622',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/74722d2d140643109bd05fc95c3fb68e/workflows.tar.gz'}


### Deploying and Testing Adversarial Patches

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [8]:
# Create Patches
response_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            f"-P data_dir={DATASET_DIR}/images/ILSVRC/Data/CLS-LOC/val-sorted-5000",
            "-P num_patch_gen_samples=10",
            "-P num_patch=1",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_train["jobId"],
)

print("Patch attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_resnet50_patch)
print("")

response_resnet50_patch = get_run_id(response_resnet50_patch)

Patch attack (ResNet50 architecture) job submitted

{'createdOn': '2021-02-14T21:13:18.482847',
 'dependsOn': '5e30b0eb-de4c-4b5c-a949-1fe044466845',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P '
                     'model=howard_imagenet_adversarial_patch_default_pretrained_resnet50/None '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000 '
                     '-P num_patch_gen_samples=10 -P num_patch=1',
 'experimentId': 20,
 'jobId': 'c765c7ef-0f3a-4038-a673-974403cf81e4',
 'lastModified': '2021-02-14T21:13:18.482847',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/db2a7028451843c8a149172808a89a47/workflows.tar.gz'}



In [9]:
# Deploy Patch attack on test set.
response_deploy_resnet50_patch_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_resnet50_patch['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            f"-P data_dir={DATASET_DIR}/images/ILSVRC/Data/CLS-LOC/val-sorted-5000",
            "-P batch_size=500",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_patch["jobId"],
)

print("Patch deployment (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_deploy_resnet50_patch_testing)
print("")

response_deploy_resnet50_patch_testing = get_run_id(response_deploy_resnet50_patch_testing)

Patch deployment (ResNet50 architecture) job submitted

{'createdOn': '2021-02-14T21:14:01.171780',
 'dependsOn': 'c765c7ef-0f3a-4038-a673-974403cf81e4',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=db570666377141f2aa28f7ea49207e4a -P '
                     'model=howard_imagenet_adversarial_patch_default_pretrained_resnet50/None '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000 '
                     '-P batch_size=500',
 'experimentId': 20,
 'jobId': 'a26bd59f-dea5-4bb8-8f48-d2a3aabd066b',
 'lastModified': '2021-02-14T21:14:01.171780',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/65c5c2c8758540c9a4d826289c306b55/workflows.tar.gz'}



In [10]:
# Check patched dataset results   
response_resnet50_infer_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_resnet50_patch_testing['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P model_architecture=resnet50",
            "-P batch_size=100",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_resnet50_patch_testing["jobId"],
)

print("Patch evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_resnet50_infer_resnet50_patch)
print("")

response_resnet50_infer_resnet50_patch = get_run_id(response_resnet50_infer_resnet50_patch)
print_mlflow_results(response_resnet50_infer_resnet50_patch)

Patch evaluation (ResNet50 architecture) job submitted

{'createdOn': '2021-02-14T21:21:28.909947',
 'dependsOn': 'a26bd59f-dea5-4bb8-8f48-d2a3aabd066b',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=24345f87b38c4278b6cb1c9a95be719f -P '
                     'model=howard_imagenet_adversarial_patch_default_pretrained_resnet50/None '
                     '-P model_architecture=resnet50 -P batch_size=100 -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset',
 'experimentId': 20,
 'jobId': '35fdd227-e6fe-4e13-b134-105c54c56056',
 'lastModified': '2021-02-14T21:21:28.909947',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/f0aa7f99e40d41fda8f67461ff9fa30c/workflows.tar.gz'}

{'accuracy': 0.5720999836921692,
 'auc': 0.9339516162872314,
 'fn': 5038.0,
 'fp': 1809.0,
 'loss': 1.9994313210344161,
 'precision': 0.7328311800956726,
 'recall': 0.4

## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [13]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the ResNet50 architecture,

In [18]:
run_resnet50 = mlflow_client.get_run(response_resnet50_infer_resnet50_patch["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [19]:
pprint.pprint(run_resnet50.data.metrics)

{'accuracy': 0.5720999836921692,
 'auc': 0.9339516162872314,
 'fn': 5038.0,
 'fp': 1809.0,
 'loss': 1.9994313210344161,
 'precision': 0.7328311800956726,
 'recall': 0.49619999527931213,
 'tn': 9988191.0,
 'tp': 4962.0}


To review the run's parameters, we use,

In [20]:
pprint.pprint(run_resnet50.data.params)

{'batch_size': '100',
 'dataset_name': 'adv_patch_dataset',
 'dataset_tar_name': 'adversarial_patch_dataset.tar.gz',
 'model': 'howard_imagenet_adversarial_patch_default_pretrained_resnet50/None',
 'model_architecture': 'resnet50',
 'run_id': '24345f87b38c4278b6cb1c9a95be719f',
 'seed': '-1'}


To review the run's tags, we use,

In [21]:
pprint.pprint(run_resnet50.data.tags)

{'mlflow.project.backend': 'securingai',
 'mlflow.project.entryPoint': 'infer',
 'mlflow.source.name': '/work/tmps38n_pty',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'securingai',
 'securingai.dependsOn': 'a26bd59f-dea5-4bb8-8f48-d2a3aabd066b',
 'securingai.jobId': '35fdd227-e6fe-4e13-b134-105c54c56056',
 'securingai.queue': 'tensorflow_gpu'}


There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).